# Task: Landlord-Controlled Feature Reconstruction

## Critical Issue Identified

**Problem**: Our current model includes features derived from guest reviews and booking history—data that doesn't exist when a landlord first posts a listing. This creates a fundamental logical flaw:

- We're predicting "value category" (Poor/Fair/Excellent)
- Value is calculated as: `FP Score = Rating / Price`
- But we're using `review_scores_rating` as a feature to predict the category
- **This is circular logic**: We were using reviews to predict a label created from reviews

## The Two-Stage Process

### Stage 1: Label Creation (Historical Data)
```
Input: Existing listings with reviews
Calculate: FP Score = review_scores_rating / price
Output: value_category (Poor/Fair/Excellent)
```

### Stage 2: Model Training (Prediction)
```
Input: ONLY landlord-controlled features (no reviews)
Predict: value_category
Goal: "Will this NEW listing be good value?"
```

## Real-World Use Case

A landlord wants to list a new property and asks:
> "Given my property features and the price I'm setting, will guests consider this good value?"

The model should answer this using ONLY information available at posting time.

---

## Step 1: Import Libraries and Load Data

We'll load the full dataset with categorical encoding to access all available features.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Create output directories
Path('../../data/processed').mkdir(parents=True, exist_ok=True)
Path('../../outputs').mkdir(parents=True, exist_ok=True)
Path('../../outputs/figures').mkdir(parents=True, exist_ok=True)

print("="*80)
print("LANDLORD-CONTROLLED FEATURE RECONSTRUCTION")
print("="*80)
print("\nLibraries loaded successfully!")

In [ ]:
# Load the full dataset
print("\nLoading dataset with categorical encoding...")
df = pd.read_csv('../../data/processed/listings_with_categorical_encoding.csv')

print(f"\nDataset loaded successfully!")
print(f"  Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nFirst few columns: {df.columns.tolist()[:10]}")

## Step 2: Categorize All Features

We'll systematically categorize every feature into three groups:

### 1.  Landlord-Controlled (Keep)
Features the landlord sets when posting the listing:
- Property characteristics (beds, bedrooms, bathrooms)
- Location (latitude, longitude, neighborhood)
- Pricing (price, minimum_nights)
- Availability settings
- Host profile information

### 2.  Review-Based (Remove)
Features that require guest reviews/bookings:
- All `review_scores_*` columns
- `number_of_reviews*` columns
- Review dates and derived features
- Estimated occupancy/revenue (requires booking history)

### 3.  Target-Related (Remove)
Features used to create or directly related to the target:
- `fp_score`, `value_category`
- `rating_normalized`, `price_normalized` (if present)

### 4.  Identifiers (Remove for modeling)
- `id`, `host_id` (keep for merging, remove for training)

In [ ]:
# Define feature categories

#  LANDLORD-CONTROLLED FEATURES
landlord_features = [
    # Property Characteristics
    'accommodates', 'bedrooms', 'beds', 'bathrooms_numeric',
    
    # Pricing (KEEP - essential for value prediction)
    'price', 'price_per_person',
    
    # Location
    'latitude', 'longitude',
    
    # Booking Rules
    'minimum_nights', 'maximum_nights',
    'minimum_minimum_nights', 'maximum_minimum_nights',
    'minimum_maximum_nights', 'maximum_maximum_nights',
    'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm',
    
    # Availability Settings
    'has_availability', 'availability_30', 'availability_60',
    'availability_90', 'availability_365', 'availability_rate',
    
    # Host Profile
    'host_response_rate', 'host_acceptance_rate',
    'host_is_superhost', 'host_listings_count',
    'host_total_listings_count', 'host_has_profile_pic',
    'host_identity_verified', 'host_years',
    
    # Booking Options
    'instant_bookable',
    
    # Categorical Encodings (from Task 1.4)
    # Room type one-hot encoded columns
    'room_type_Hotel room', 'room_type_Private room', 'room_type_Shared room',
    
    # Property type encodings
    'property_type_label', 'property_type_frequency',
    
    # Neighborhood encodings
    'neighbourhood_target_encoded', 'neighbourhood_frequency',
    'neighbourhood_label',
    
    # Host response time (one-hot encoded)
    'host_response_time_within a day',
    'host_response_time_within a few hours',
    'host_response_time_within an hour',
    
    # Calculated host listings
    'calculated_host_listings_count',
    'calculated_host_listings_count_entire_homes',
    'calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms'
]

#  REVIEW-BASED FEATURES (Remove)
review_features = [
    # Review Scores
    'review_scores_rating', 'review_scores_accuracy',
    'review_scores_cleanliness', 'review_scores_checkin',
    'review_scores_communication', 'review_scores_location',
    'review_scores_value', 'avg_review_score',
    
    # Review Counts
    'number_of_reviews', 'number_of_reviews_ltm',
    'number_of_reviews_l30d', 'reviews_per_month',
    'has_reviews',
    
    # Review Dates
    'first_review', 'last_review',
    'days_since_first_review', 'days_since_last_review',
    
    # Booking History
    'estimated_occupancy_l365d', 'estimated_revenue_l365d',
    'availability_eoy', 'number_of_reviews_ly'
]

#  TARGET-RELATED FEATURES (Remove)
target_features = [
    'fp_score', 'value_category', 'value_encoded',
    'rating_normalized', 'price_normalized'
]

# 🗑️ IDENTIFIER COLUMNS (Keep for merging, remove for training)
id_columns = ['id', 'host_id']

print("\n" + "="*80)
print("FEATURE CATEGORIZATION")
print("="*80)
print(f"\n Landlord-Controlled Features: {len(landlord_features)}")
print(f" Review-Based Features: {len(review_features)}")
print(f"  Target-Related Features: {len(target_features)}")
print(f"  Identifier Columns: {len(id_columns)}")

## Step 3: Verify Feature Availability

Check which features from our categorization actually exist in the dataset.

In [ ]:
# Check which features exist in the dataset
available_landlord = [f for f in landlord_features if f in df.columns]
available_review = [f for f in review_features if f in df.columns]
available_target = [f for f in target_features if f in df.columns]
available_ids = [f for f in id_columns if f in df.columns]

print("\n" + "="*80)
print("FEATURE AVAILABILITY CHECK")
print("="*80)

print(f"\n Available Landlord Features: {len(available_landlord)}/{len(landlord_features)}")
missing_landlord = set(landlord_features) - set(available_landlord)
if missing_landlord:
    print(f"   Missing: {missing_landlord}")

print(f"\n Available Review Features: {len(available_review)}/{len(review_features)}")
print(f"   These will be REMOVED from the model")

print(f"\n  Available Target Features: {len(available_target)}/{len(target_features)}")
print(f"   These will be REMOVED from features (kept as target)")

print(f"\n  Available ID Columns: {len(available_ids)}/{len(id_columns)}")
print(f"   These will be kept for merging but removed for training")

## Step 4: Create Clean Feature Set

Now we'll create the final dataset with:
1. All landlord-controlled features
2. ID columns (for merging with NLP features later)
3. Target variable (`value_category`)
4. NO review-based features

In [ ]:
# Create the clean feature set
features_to_keep = available_ids + available_landlord + ['value_category']

# Filter to only existing columns
features_to_keep = [f for f in features_to_keep if f in df.columns]

# Create clean dataset
df_clean = df[features_to_keep].copy()

print("\n" + "="*80)
print("CLEAN DATASET CREATED")
print("="*80)
print(f"\nOriginal dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Clean dataset: {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns")
print(f"\nFeatures removed: {df.shape[1] - df_clean.shape[1]}")
print(f"Features retained: {df_clean.shape[1] - 1} (excluding target)")

# Check for missing values
missing_counts = df_clean.isnull().sum()
cols_with_missing = missing_counts[missing_counts > 0]

if len(cols_with_missing) > 0:
    print(f"\n  Warning: {len(cols_with_missing)} columns have missing values")
    print("\nTop 10 columns with missing values:")
    print(cols_with_missing.sort_values(ascending=False).head(10))
else:
    print("\n No missing values in the clean dataset!")

## Step 5: Feature Summary Statistics

Let's examine the distribution of our landlord-controlled features.

In [ ]:
# Separate numerical and categorical features
numerical_features = df_clean.select_dtypes(include=[np.number]).columns.tolist()
numerical_features = [f for f in numerical_features if f not in available_ids + ['value_category']]

print("\n" + "="*80)
print("NUMERICAL FEATURES SUMMARY")
print("="*80)
print(f"\nTotal numerical features: {len(numerical_features)}")

# Display summary statistics
summary_stats = df_clean[numerical_features].describe().T
summary_stats['missing'] = df_clean[numerical_features].isnull().sum()
summary_stats['missing_pct'] = (summary_stats['missing'] / len(df_clean) * 100).round(2)

print("\nKey features summary:")
key_features = ['price', 'accommodates', 'bedrooms', 'beds', 'bathrooms_numeric',
                'availability_365', 'minimum_nights', 'host_years']
key_features = [f for f in key_features if f in numerical_features]

print(summary_stats.loc[key_features, ['mean', 'std', 'min', '50%', 'max', 'missing']].to_string())

## Step 6: Visualize Feature Distributions

Create visualizations to understand our landlord-controlled features.

In [ ]:
# Create visualization of key features
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

key_viz_features = ['price', 'accommodates', 'bedrooms', 'beds', 
                    'bathrooms_numeric', 'availability_365']
key_viz_features = [f for f in key_viz_features if f in df_clean.columns]

for idx, feature in enumerate(key_viz_features):
    ax = axes[idx]
    
    # Filter outliers for better visualization
    data = df_clean[feature].dropna()
    q99 = data.quantile(0.99)
    data_filtered = data[data <= q99]
    
    ax.hist(data_filtered, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    ax.set_xlabel(feature.replace('_', ' ').title(), fontsize=11)
    ax.set_ylabel('Frequency', fontsize=11)
    ax.set_title(f'{feature.replace("_", " ").title()} Distribution', fontsize=12, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # Add mean line
    mean_val = data_filtered.mean()
    ax.axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.1f}')
    ax.legend(fontsize=9)

plt.suptitle('Landlord-Controlled Features Distribution', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig('../../outputs/figures/landlord_features_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n Saved: outputs/figures/landlord_features_distribution.png")

## Step 7: Target Variable Distribution

Verify that our target variable is still balanced after feature filtering.

In [ ]:
# Check target distribution
target_dist = df_clean['value_category'].value_counts().sort_index()
target_pct = (target_dist / len(df_clean) * 100).round(2)

print("\n" + "="*80)
print("TARGET VARIABLE DISTRIBUTION")
print("="*80)
print(f"\nValue Category Distribution:")
for category, count in target_dist.items():
    pct = target_pct[category]
    print(f"  {category}: {count:,} ({pct}%)")

# Visualize target distribution
fig, ax = plt.subplots(figsize=(10, 6))
colors = {'Excellent_Value': '#2ecc71', 'Fair_Value': '#f1c40f', 'Poor_Value': '#e74c3c'}
bars = ax.bar(target_dist.index, target_dist.values, 
              color=[colors.get(x, 'gray') for x in target_dist.index],
              edgecolor='black', linewidth=1.5)

# Add percentage labels
for bar, pct in zip(bars, target_pct.values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{pct}%',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_xlabel('Value Category', fontsize=13, fontweight='bold')
ax.set_ylabel('Count', fontsize=13, fontweight='bold')
ax.set_title('Target Variable Distribution (Landlord-Controlled Features Only)', 
             fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('../../outputs/figures/target_distribution_clean.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n Saved: outputs/figures/target_distribution_clean.png")

## Step 8: Save Clean Dataset

Save the cleaned dataset with only landlord-controlled features for model training.

In [ ]:
# Save the clean dataset
output_path = '../../data/processed/listings_landlord_features_only.csv'
df_clean.to_csv(output_path, index=False)

print("\n" + "="*80)
print("DATASET SAVED")
print("="*80)
print(f"\n✅ Saved: {output_path}")
print(f"   Shape: {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns")
print(f"   Features (excluding ID and target): {df_clean.shape[1] - len(available_ids) - 1}")

# Save feature list for reference
feature_list = [f for f in df_clean.columns if f not in available_ids + ['value_category']]
feature_df = pd.DataFrame({
    'feature_name': feature_list,
    'data_type': [df_clean[f].dtype for f in feature_list],
    'missing_count': [df_clean[f].isnull().sum() for f in feature_list],
    'missing_pct': [(df_clean[f].isnull().sum() / len(df_clean) * 100).round(2) for f in feature_list]
})

feature_list_path = '../../outputs/landlord_features_list.csv'
feature_df.to_csv(feature_list_path, index=False)
print(f"\n Saved feature list: {feature_list_path}")

## Step 9: Create Train-Test Split

Split the clean dataset for model training, maintaining the same 80-20 split with stratification.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle

# Separate features and target
X = df_clean.drop(columns=['value_category'] + available_ids)
y = df_clean['value_category']
ids = df_clean[available_ids]

# Train-test split (80-20, stratified)
X_train, X_test, y_train, y_test, ids_train, ids_test = train_test_split(
    X, y, ids, test_size=0.2, random_state=42, stratify=y
)

print("\n" + "="*80)
print("TRAIN-TEST SPLIT")
print("="*80)
print(f"\nTraining set: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set: {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"Number of features: {X_train.shape[1]}")

print(f"\nTraining set class distribution:")
print(y_train.value_counts().sort_index())

print(f"\nTest set class distribution:")
print(y_test.value_counts().sort_index())

## Step 10: Feature Scaling

Apply StandardScaler to normalize features for algorithms sensitive to feature scales.

In [ ]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame for easier handling
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("\n" + "="*80)
print("FEATURE SCALING")
print("="*80)
print(f"\n Features scaled using StandardScaler")
print(f"   Training set shape: {X_train_scaled_df.shape}")
print(f"   Test set shape: {X_test_scaled_df.shape}")

# Verify scaling
print(f"\nScaling verification (training set):")
print(f"   Mean (should be ~0): {X_train_scaled_df.mean().mean():.6f}")
print(f"   Std (should be ~1): {X_train_scaled_df.std().mean():.6f}")

## Step 11: Save Processed Data

Save all processed datasets for model training.

In [ ]:
# Save unscaled data
X_train.to_csv('../../data/processed/X_train_landlord.csv', index=False)
X_test.to_csv('../../data/processed/X_test_landlord.csv', index=False)
y_train.to_csv('../../data/processed/y_train_landlord.csv', index=False, header=True)
y_test.to_csv('../../data/processed/y_test_landlord.csv', index=False, header=True)

# Save scaled data
X_train_scaled_df.to_csv('../../data/processed/X_train_scaled_landlord.csv', index=False)
X_test_scaled_df.to_csv('../../data/processed/X_test_scaled_landlord.csv', index=False)

# Save IDs for merging with NLP features later
ids_train.to_csv('../../data/processed/train_ids.csv', index=False)
ids_test.to_csv('../../data/processed/test_ids.csv', index=False)

# Save scaler
with open('../../models/scaler_landlord.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("\n" + "="*80)
print("ALL FILES SAVED")
print("="*80)
print("\n Unscaled Data:")
print("    data/processed/X_train_landlord.csv")
print("    data/processed/X_test_landlord.csv")
print("    data/processed/y_train_landlord.csv")
print("    data/processed/y_test_landlord.csv")
print("\n Scaled Data:")
print("    data/processed/X_train_scaled_landlord.csv")
print("    data/processed/X_test_scaled_landlord.csv")
print("\n Supporting Files:")
print("    data/processed/train_ids.csv (for NLP merging)")
print("    data/processed/test_ids.csv (for NLP merging)")
print("    models/scaler_landlord.pkl")
print("    outputs/landlord_features_list.csv")

## Step 12: Summary Report

Generate a comprehensive summary of the feature reconstruction process.

In [ ]:
print("\n" + "="*80)
print("LANDLORD-CONTROLLED FEATURE RECONSTRUCTION - SUMMARY")
print("="*80)

summary_report = f"""
## Problem Identified
Original model included review-based features (review_scores_rating, 
number_of_reviews, etc.) which are not available when a landlord first 
posts a listing. This created circular logic since these features were 
used to predict labels created from those same features.

## Solution Implemented
Rebuilt feature set to include ONLY landlord-controlled features:
- Property characteristics (beds, bedrooms, bathrooms, accommodates)
- Location (latitude, longitude, neighborhood encodings)
- Pricing (price, price_per_person) ← KEPT as essential
- Booking rules (minimum_nights, availability settings)
- Host profile (host_years, superhost status, response rates)

## Data Summary
Original dataset: {df.shape[0]:,} rows × {df.shape[1]} columns
Clean dataset: {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns
Features removed: {df.shape[1] - df_clean.shape[1]}
Features retained: {X_train.shape[1]} (for modeling)

Training samples: {X_train.shape[0]:,} ({X_train.shape[0]/len(X)*100:.1f}%)
Test samples: {X_test.shape[0]:,} ({X_test.shape[0]/len(X)*100:.1f}%)

## Target Distribution (Balanced)
{y.value_counts().sort_index().to_string()}

## Key Features Included
 price (essential for value prediction)
 accommodates, bedrooms, beds, bathrooms_numeric
 latitude, longitude, neighborhood encodings
 availability_365, minimum_nights
 host_years, host_is_superhost
✅room_type encodings, property_type encodings

## Features REMOVED (Review-Based)
 review_scores_* (all review score columns)
 number_of_reviews* (all review count columns)
 estimated_occupancy_l365d, estimated_revenue_l365d
 first_review, last_review, days_since_*_review

## Expected Model Performance
Previous accuracy with reviews: ~99% (circular logic)
Expected accuracy without reviews: ~65-75% (realistic)

The lower accuracy is CORRECT because:
1. We're predicting future value without knowing future reviews
2. The model must learn from property features alone
3. This matches real-world use case (new listing prediction)

## Next Steps
1. Retrain all models (Logistic Regression, Random Forest, XGBoost, SVM, MLP)
2. Compare performance with landlord-only features
3. Add NLP features from listing descriptions (optional enhancement)
4. Evaluate realistic model performance

## Files Generated
 listings_landlord_features_only.csv - Full clean dataset
 X_train_landlord.csv, X_test_landlord.csv - Unscaled splits
 X_train_scaled_landlord.csv, X_test_scaled_landlord.csv - Scaled splits
 y_train_landlord.csv, y_test_landlord.csv - Target variables
 train_ids.csv, test_ids.csv - IDs for NLP merging
 scaler_landlord.pkl - Fitted scaler for deployment
 landlord_features_list.csv - Feature documentation
"""

print(summary_report)

# Save summary report
with open('../../outputs/reports/landlord_features_reconstruction_summary.txt', 'w') as f:
    f.write(summary_report)

print("\n Summary saved: outputs/landlord_features_reconstruction_summary.txt")
print("\n" + "="*80)
print("TASK COMPLETE - Ready for Model Retraining")
print("="*80)

---

## Conclusion

We have successfully reconstructed our feature set to include only landlord-controlled variables. This ensures our model can make realistic predictions for NEW listings where review data doesn't exist yet.

### Key Takeaways

1. **Price is NOT leakage** - It's an essential input that landlords control
2. **Review features ARE leakage** - They don't exist for new listings
3. **Lower accuracy is expected** - We're solving a harder, more realistic problem
4. **Model is now deployable** - Can predict value for any new listing

### Real-World Application

A landlord can now input:
- Property details (beds, bathrooms, location)
- Desired price
- Booking rules

And receive a prediction:
> "Based on similar properties in your area, this listing will likely be considered **Fair Value** by guests."

This is actionable intelligence that helps landlords optimize their pricing strategy BEFORE receiving any reviews.